In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hlsf.models import LSF_DATA
from scipy import interpolate
from astropy.io import fits
%matplotlib qt

In [2]:
config = 'H'
lamp = 'Kr'

In [3]:
lsf_data = LSF_DATA(f"../exposures/ARC-{lamp}_CLEAR_20MAS_{config}_PRM.fits", f"../text/{lamp}.txt", f"../exposures/WAVECAL_TABLE_20MAS_{config}.fits", 
                        f"../exposures/SLITLET_TABLE_20MAS_{config}.fits", slice=10, normal='Normal')
lsf_data.get_line_list()

{171: 14402.22,
 172: 14426.793,
 173: 14517.84,
 174: 14734.436,
 177: 14961.894,
 178: 15005.307,
 179: 15209.526,
 180: 15239.615,
 183: 15372.037,
 184: 15474.026,
 185: 15681.02,
 186: 15820.09,
 187: 16726.513,
 188: 16785.128,
 189: 16853.488,
 192: 16935.806,
 193: 17098.771,
 194: 17367.606,
 195: 17404.443,
 196: 17616.854,
 197: 17842.737,
 198: 18002.229,
 199: 18167.315,
 200: 18399.786}

In [5]:
lsf_data.write_fits('../images/image.fits')

In [8]:
hdul = fits.open('../images/image.fits')
hdul.info()

Filename: ../images/image.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      53   ()      
  1  INTENSITY     1 ImageHDU         8   (818, 80)   float64   
  2  X.COORDINATE    1 TableHDU        12   818R x 1C   [D25.17]   
  3  Y.COORDINATE    1 TableHDU        12   80R x 1C   [D25.17]   
